In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
sys.path.append('../helper_functions')
sys.path.append('../objects')
# sys.path.append('..')

import copy
import time
import dill
import numpy as np
import scipy.io as io
import matplotlib.pyplot as plt
from scipy.spatial.transform import Rotation as R
import plotly.graph_objects as go

%matplotlib inline
# %matplotlib qt

In [4]:
from prefixes import remove_prefix, add_prefix
from conversions import rho24sig, convert2mainSI

from ElectricObject import ElectricObject

from Worm import Worm
from Worm_SmallSpherical import SmallSphericalWorm

from Fish import Fish
from FishGeneration import FishGeneration
from Fish_IceCreamCone import IceCreamConeFish
from Fish_Capsule import CapsuleFish


from Boundaries import *
from Aquarium import Aquarium
from Aquarium_SinglePlane import SinglePlaneAquarium

from assertion_tests import run_tests
run_tests()

'Success!'

## Load and process the EOD waveform

In [5]:
full_eod = io.loadmat('../../eods/eod_fit.mat')['fitted_eod'].flatten()

# erase the 0-padding around the eod for efficiency
max_eod = np.abs(full_eod).max()
alt_0 = 1e-2
good_id_bounds = np.where(np.abs(full_eod) > max_eod * alt_0)[0]
full_eod = full_eod[good_id_bounds[0]:good_id_bounds[-1]]
full_eod-= full_eod[0]
full_eod.shape

(1138,)

## Load the receptors filters

In [6]:
# conv_filters = dill.load(open('../../conv_filters/conv_filters_from_data.pkl', 'rb'))
# plt.plot(conv_filters.T)
# conv_filters = dill.load(open('../../conv_filters/conv_filters_PCA.pkl', 'rb'))
# plt.plot(conv_filters.T)
conv_filters = dill.load(open('../../conv_filters/conv_filters_from_data.pkl', 'rb'))
conv_filters.shape

(2, 1138)

## Check the input arguments for the wanted fish object

In [7]:
# for x in CapsuleFishGeneration._initialize_input_argument_names():
#     print(f'{x},')

## Create the Fish object

In [8]:
# # # position = (0.150-0.020, 0.007, 0.000)
# # # square_grid = np.linspace(-0.015,0.015,9)
# # # points = np.array(position).reshape(-1,3)
# # # points = np.vstack([points, np.vstack(np.meshgrid(position[0] - square_grid, [position[1]], position[2] - square_grid)).reshape(3,-1).T])
# # # points = np.vstack([points, np.vstack(np.meshgrid(position[0] - square_grid/10, [position[1]], position[2] - square_grid/10)).reshape(3,-1).T])

# # define the base fish
# base_fish = CapsuleFish(
#     # receptors_locations=np.array([]).reshape(0,3),
#     # receptors_normals=np.array([]).reshape(0,3),
#     # point_currents_magnitudes=np.array([]).reshape(0),
#     # point_currents_locations=np.array([]).reshape(0,3),
#     receptor_filters=conv_filters,
#     eod_waveform=full_eod,
#     skin_resistivity=3 * 1e3 * (1e-2)**2,
#     sampling_rate=(2.5, "M"),  # MHz,
#     eod_delay=(0,), # s,
#     nose_position=[0,0,0],
#     fish_length=(15, 'c'),
#     angle_yaw=(0, 'deg'),
#     angle_pitch=(0, 'deg'),
#     angle_roll=(0, 'deg'),
#     relative_bend_locations_percentage=np.array([]),
#     relative_bend_angle_lateral=(np.array([]),"deg"),
#     relative_bend_angle_dorso_ventral =(np.array([]),"deg"),
#     point_currents_range_percentage=dict(start=0, end=100),
#     N_point_currents=101,
#     point_currents_magnitude_scale=1,
#     receptors_init=dict(method='grid_uniDense',
#                         head_t=0, head_u=0,
#                         tail_t=0, tail_u=0,
#                         body_t=60, body_u=30,),
#     # receptors_init=dict(method='manual',
#     #                     points=points,
#     #                     normals=np.tile(np.array([[0,1,0]]), (points.shape[0],1))),
#     vertical_semi_axis=(2.0, 'c'),  # (1.4, 'c'),
#     lateral_semi_axis=(0.9, 'c'),
#     rostrocaudal_semi_axis=0, #None,
#     rostrocaudal_semi_axis_tail=0,#(3, 'c'),
# )      

# receptors_locations = base_fish.get_receptors_locations()
# receptors_normals   = base_fish.get_receptors_normals()

# receptors_normals   = receptors_normals  [receptors_locations[:,0]>base_fish.get_length() * (3/7-1),:]
# receptors_locations = receptors_locations[receptors_locations[:,0]>base_fish.get_length() * (3/7-1),:]

# # receptors_normals   = receptors_normals  [receptors_locations[:,1]>=0,:]
# # receptors_locations = receptors_locations[receptors_locations[:,1]>=0,:]

# # receptors_normals   = receptors_normals  [np.linalg.norm(receptors_locations[:,:2], axis=1) > 1e-6,:]
# # receptors_locations = receptors_locations[np.linalg.norm(receptors_locations[:,:2], axis=1) > 1e-6,:]

# new_order = np.lexsort((-receptors_locations[:,0], -receptors_locations[:,2]))
# receptors_normals   = receptors_normals[new_order]
# receptors_locations = receptors_locations[new_order]


# base_fish.update_receptors(receptors_locations=receptors_locations, receptors_normals=receptors_normals)

# dill.dump(base_fish, open('../data/baseFish/data_240425_Base_Detection_1050_receptors.pkl', 'wb'), protocol=4)
# base_fish = dill.load(open('../data/baseFish/data_220623_Base_fish_with_Single_Receptor.pkl', 'rb'))
# base_fish = dill.load(open('../data/baseFish/data_220610_base_fish_with_receptor_responses.pkl', 'rb'))
# base_fish = dill.load(open('../data/baseFish/data_230119_Base_ImgConv_fish.pkl', 'rb'))
# base_fish = dill.load(open('../data/baseFish/data_230123_Base_ImgConv_fish.pkl', 'rb'))
# base_fish = dill.load(open('../data/baseFish/data_231109_Base_SpatialProcessTest_fish.pkl', 'rb'))
# base_fish = dill.load(open('../data/baseFish/data_231115_Base_SpatialProcess_MovingObject_fish.pkl', 'rb'))
# base_fish = dill.load(open('../data/baseFish/data_231116_Base_SpatialProcess_MovingObject_fish.pkl', 'rb'))
# base_fish = dill.load(open('../data/baseFish/data_240411_Base_SpatialProcess_MovingObject_fish.pkl', 'rb'))
base_fish = dill.load(open('../data/baseFish/data_240425_Base_Detection_1050_receptors.pkl', 'rb'))


In [9]:
base_fish.visualize_scatter(show_normals=2, show_point_currents=3)
print(base_fish.get_receptors_locations().shape)

(1050, 3)


### Test runtime of changing fish parameters

In [10]:
# t1 = time.time()
# base_fish.update_parameters(
#     nose_position=[0.15,0,0],
#     angle_yaw=(0, 'deg'),
#     angle_pitch=(0, 'deg'),
#     angle_roll=(0, 'deg'),
#     relative_bend_locations_percentage=np.array([60]),
#     relative_bend_angle_lateral=(np.array([-60]),"deg"),
#     relative_bend_angle_dorso_ventral =(np.array([0]),"deg"),
#     point_currents_range_percentage=dict(start=15, end=95),
#     N_point_currents=201,
#     point_currents_magnitude_scale=1,
# )
# t2 = time.time()
# base_fish.visualize_scatter(show_normals=2, show_point_currents=3)
# t3 = time.time()
# print(f'{t2-t1:.5f}, {t3-t2:.5f}')

In [11]:
# Create Data Params Dict
data_params_dict = {}
# data_params_dict['save_file_name']                  = 'data_240411_SpatialProcessing_MovingObject_7by15'
data_params_dict['save_file_name']                  = 'data_240425_Detection_1050_receptors_HighContrast'
# aquarium properties
data_params_dict['water_conductivities']            = [0.01]
data_params_dict['boundary_displacements']          = [[-1e9]]
data_params_dict['boundary_normals']                =  [[0,0,1]]
# fish properties
data_params_dict['fish_obj']                        = base_fish
data_params_dict['fish_bend_angle_lateral']         = [[0]]
data_params_dict['fish_bend_angle_dorso_ventral']   = [[0]]
data_params_dict['fish_bend_location_percentages']  = [[0]]
data_params_dict['fish_yaw']                        = [0]
data_params_dict['fish_pitch']                      = [0]
data_params_dict['fish_roll']                       = [0]
# worm properties
# data_params_dict['worm_resistances']                = np.exp(np.linspace(np.log(2), np.log(100), 5)) * 1e3
data_params_dict['worm_resistances']                = np.array([2e3, 100e3])  # HighContrast
data_params_dict['worm_capacitances']               = np.exp(np.linspace(np.log(2), np.log(100), 5)) * 1e-10  # HighContrast when paired with HighContrast resistances
# data_params_dict['worm_capacitances']               = np.array([2e-10])  # HighContrast
data_params_dict['worm_radii']                      = np.array([12.5]) * 1e-3
data_params_dict['worm_position_xs']                = np.array([-30]) * 1e-3   # - np.arange(101) * 3e-4
data_params_dict['worm_position_ys']                = (5 * np.power(np.sqrt(1.33), np.arange(25))).round(0) * 1e-3 + base_fish.lat_ax + data_params_dict['worm_radii']
data_params_dict['worm_position_zs']                = np.array([0]) * 1e-3
# save params file
dill.dump(data_params_dict, open('../data/params/' + data_params_dict['save_file_name'] + '_params_dict.pkl', 'wb'), protocol=4)

In [ ]:
# Create Data Params Dict
data_params_dict = {}
data_params_dict['save_file_name']                  = 'data_231116_SpatialProcessing_MovingObject_11by25'
# aquarium properties
data_params_dict['water_conductivities']            = [0.01]  # np.array([100, 300, 600]) * 1e-6 / 1e-2
data_params_dict['boundary_displacements']          = [[-1e9]]
                                                      # [[-1e9]+list(-np.arange(35, 70, 5)[::-1]*1e-3),
                                                      #  [-1e9]+list(-np.arange(35, 70, 5)[::-1]*1e-3), 
                                                      #  [-1e9]+list(-np.arange(35, 70, 5)[::-1]*1e-3)]
                                                    #  [[-1e9]+list(-np.linspace(5*base_fish.roc_ax, base_fish.roc_ax, 9)), ]  # 
                                                    #    [-1e9]+list(-np.linspace(0.058, 0.022, 9)),
                                                    #    [-1e9]+list(-np.linspace(0.058, 0.022, 9))]
data_params_dict['boundary_normals']                =  [[0,0,1]]
                                                      # [[0,0,1], [0,-1,0], [0,1,0]]
                                                    #  [[0,0,1], ]  #
                                                    #    [0,-1,0],
                                                    #    [0,1,0]]
# fish properties
data_params_dict['fish_obj']                        = base_fish
data_params_dict['fish_bend_angle_lateral']         = [[0]]#[[x] for x in np.linspace(-60,60,25)]
data_params_dict['fish_bend_angle_dorso_ventral']   = [[0]]#[[0]]
data_params_dict['fish_bend_location_percentages']  = [[0]]#[[60]]
data_params_dict['fish_yaw']                        = [0]
data_params_dict['fish_pitch']                      = [0]
data_params_dict['fish_roll']                       = [0]
# worm properties
data_params_dict['worm_resistances']                = np.exp(np.linspace(np.log(2), np.log(100), 5)) * 1e3
data_params_dict['worm_capacitances']               = np.exp(np.linspace(np.log(2), np.log(100), 5)) * 1e-10
data_params_dict['worm_radii']                      = np.array([12.5]) * 1e-3
data_params_dict['worm_position_xs']                = np.array([-30]) * 1e-3 - np.arange(11) * 3e-3
data_params_dict['worm_position_ys']                = np.array([5,10,15,30,50,70,100,150]) * 1e-3 + base_fish.lat_ax + data_params_dict['worm_radii']
data_params_dict['worm_position_zs']                = np.array([0]) * 1e-3
# save params file
dill.dump(data_params_dict, open('../data/params/' + data_params_dict['save_file_name'] + '_params_dict.pkl', 'wb'), protocol=4)

In [ ]:
# Create Data Params Dict
data_params_dict = {}
data_params_dict['save_file_name']                  = 'data_240311_falloff_with_distance_and_radius'
# aquarium properties
data_params_dict['water_conductivities']            = np.array([100]) * 1e-6 / 1e-2
data_params_dict['boundary_displacements']          = [[-1e9]]
data_params_dict['boundary_normals']                =  [[0,0,1]]
# fish properties
data_params_dict['fish_obj']                        = base_fish
data_params_dict['fish_bend_angle_lateral']         = [[0]]
data_params_dict['fish_bend_angle_dorso_ventral']   = [[0]]
data_params_dict['fish_bend_location_percentages']  = [[0]]
data_params_dict['fish_yaw']                        = [0]
data_params_dict['fish_pitch']                      = [0]
data_params_dict['fish_roll']                       = [0]
# worm properties
data_params_dict['worm_resistances']                = np.array([500,50000])
data_params_dict['worm_capacitances']               = np.array([2e-10])
data_params_dict['worm_radii']                      = np.arange(5,11) * 1e-3
data_params_dict['worm_position_xs']                = np.array([-30]) * 1e-3
data_params_dict['worm_position_ys']                = np.arange(10,201) * 1e-3 + base_fish.lat_ax
data_params_dict['worm_position_zs']                = np.array([0]) * 1e-3
# save params file
dill.dump(data_params_dict, open('../data/params/' + data_params_dict['save_file_name'] + '_params_dict.pkl', 'wb'), protocol=4)

In [ ]:
# # Create Data Params Dict
# data_params_dict = {}
# data_params_dict['save_file_name']                  = 'data_230123_SMALL_YES_manyObjects_NO_boundaries_NO_tail_ONE_conductivity'
# # aquarium properties
# data_params_dict['water_conductivities']            = np.array([100]) * 1e-6 / 1e-2
# data_params_dict['boundary_displacements']          = [[-1e9]]
# data_params_dict['boundary_normals']                =  [[0,0,1]]
# # fish properties
# data_params_dict['fish_obj']                        = base_fish
# data_params_dict['fish_bend_angle_lateral']         = [[0]]
# data_params_dict['fish_bend_angle_dorso_ventral']   = [[0]]
# data_params_dict['fish_bend_location_percentages']  = [[0]]
# data_params_dict['fish_yaw']                        = [0]
# data_params_dict['fish_pitch']                      = [0]
# data_params_dict['fish_roll']                       = [0]
# # worm properties
# data_params_dict['worm_resistances']                = np.exp(np.linspace(np.log(1), np.log(1000), 10)) * 1e2
# data_params_dict['worm_capacitances']               = np.exp(np.linspace(np.log(1), np.log(1000), 10)) * 1e-10
# data_params_dict['worm_radii']                      = np.array([1, 3, 5, 7]) * 1e-3
# data_params_dict['worm_position_xs']                = np.array([5, -15, -35, -55]) * 1e-3
# data_params_dict['worm_position_ys']                = np.array([8, 12, 16, 20]) * 1e-3 + base_fish.lat_ax
# data_params_dict['worm_position_zs']                = np.array([-10, 0, 10]) * 1e-3
# # save params file
# dill.dump(data_params_dict, open('../data/params/' + data_params_dict['save_file_name'] + '_params_dict.pkl', 'wb'), protocol=4)

# Modify previous parameters for re-runs

In [ ]:
# data_params_dict = dill.load(open('../data/params/data_210715_dataset_3receptors_4sigs_3ys_many_R_C_params_dict.pkl','rb'))
# fish = data_params_dict['fish_obj']
# data_params_dict = dill.load(open('../data/params/data_210317_dataset_3receptors_3sigs_3ys_many_R_C_params_dict.pkl','rb'))
# data_params_dict = dill.load(open('../data/params/data_210721_dataset_3receptors_3sigs_3ys_many_R_C_CONTROL_SIMULATIONS_MODIFIED_params_dict.pkl','rb'))
# data_params_dict = dill.load(open('../data/params/data_210630_dataset_3receptors_3sigs_3ys_many_R_C_params_dict.pkl','rb'))
# data_params_dict = dill.load(open('../data/params/data_220304_singleReceptor_Optimize_Feature_Space_params_dict.pkl','rb'))
data_params_dict = dill.load(open('../data/params/data_220610_Large_dataset_with_receptors_responses_params_dict.pkl','rb'))
# data_params_dict = dill.load(open('../data/params/data_220308_PCA_of_RC_effects_params_dict.pkl','rb'))


In [ ]:
data_params_dict

In [ ]:
# GENERATE DATA WITH DIFFERENT DISTRIBUTIONS FOR PCA OF LEOD
# UNIFORM GRID IN LOG R & C
# resistances = np.exp(np.linspace(np.log(20), np.log(1000), 200)) * 1e2
# capacitances = np.exp(np.linspace(np.log(2), np.log(1000), 200)) * 1e-10

# UNIFORM RANDOM IN LOG R & C
# resistances = np.exp(np.sort(np.random.rand(200) * (np.log(1000) - np.log(20)) + np.log(20))) * 1e2
# capacitances = np.exp(np.sort(np.random.rand(200) * (np.log(1000) - np.log(2)) + np.log(2))) * 1e-10

# UNIFORM RANDOM IN R & C
# resistances = np.sort(np.random.rand(200) * (100000 - 2000) + 2000)
# capacitances = np.sort(np.random.rand(200) * (1000e-10 - 2e-10) + 2e-10)

# GAUSSIAN RANDOM IN R & C WITH MEAN AROUND MOST VARIABLE POINT IN FEATURE SPACE
# resistances = np.sort(np.random.randn(200) * data_params_dict['worm_resistances'][::8][9-3:9+3].std() + data_params_dict['worm_resistances'][::8][9-3:9+3].mean())
# capacitances = np.sort(np.random.randn(200) * data_params_dict['worm_capacitances'][::8][12-3:12+3].std() + data_params_dict['worm_capacitances'][::8][12-3:12+3].mean())

# SECOND (LARGER STD) GAUSSIAN RANDOM IN R & C WITH MEAN AROUND MOST VARIABLE POINT IN FEATURE SPACE
resistances = np.sort(np.abs(np.random.randn(200) * data_params_dict['worm_resistances'][::8][9-3:9+3].std() * 2 + data_params_dict['worm_resistances'][::8][9-3:9+3].mean()))
capacitances = np.sort(np.abs(np.random.randn(200) * data_params_dict['worm_capacitances'][::8][12-3:12+3].std() * 2 + data_params_dict['worm_capacitances'][::8][12-3:12+3].mean()))

data_params_dict['worm_resistances'] = resistances
data_params_dict['worm_capacitances'] = capacitances
data_params_dict['fish_obj'] = base_fish
data_params_dict['worm_position_xs'] = [-0.03]
data_params_dict['save_file_name'] = 'data_221220_PCA_of_RC_effects_Gauss_Wide_Rand_Dist'

In [ ]:
# plt.plot(resistances)
plt.plot(capacitances)

In [ ]:
# REPAIR SOME RUN

# data_params_dict['save_file_name'] = 'repair_1'
# data_params_dict['water_conductivities'] = [0.06]
# data_params_dict['worm_capacitances'] = [data_params_dict['worm_capacitances'][24]]
# data_params_dict['worm_resistances'] = data_params_dict['worm_resistances'][15:]


In [ ]:
data_params_dict

In [ ]:
dill.dump(data_params_dict, open('../data/params/'+data_params_dict['save_file_name'] + '_params_dict.pkl', 'wb'), protocol=4)

# Remake the 220610 run to check the features "sign"
### Because I was looking and the wrong receptor

In [ ]:
dataset = dill.load(open('../data/processed/data_220610_Large_dataset_with_receptors_responses.pkl', 'rb'))

In [ ]:
data_params_dict = {}
data_params_dict['save_file_name']                  = 'data_230315_Large_dataset_with_receptors_responses_REMAKE'
# aquarium properties
data_params_dict['water_conductivities']            = dataset['aqua']['conductivity'][:1]
data_params_dict['boundary_displacements']          = [[dataset['aqua']['boundary_displacements'][0][0]]]
data_params_dict['boundary_normals']                = dataset['aqua']['boundary_normals']
# fish properties
data_params_dict['fish_obj']                        = dataset['fish']['fish_objs'][6]
data_params_dict['fish_bend_angle_lateral']         = [[0]]
data_params_dict['fish_bend_angle_dorso_ventral']   = [[0]]
data_params_dict['fish_bend_location_percentages']  = [[0]]
data_params_dict['fish_yaw']                        = [0]
data_params_dict['fish_pitch']                      = [0]
data_params_dict['fish_roll']                       = [0]
# worm properties
data_params_dict['worm_resistances']                = dataset['worms']['resistances']
data_params_dict['worm_capacitances']               = dataset['worms']['capacitances']
data_params_dict['worm_radii']                      = dataset['worms']['radii']
data_params_dict['worm_position_xs']                = dataset['worms']['position_xs']
data_params_dict['worm_position_ys']                = dataset['worms']['position_ys']
data_params_dict['worm_position_zs']                = dataset['worms']['position_zs']
# save params file
dill.dump(data_params_dict, open('../data/params/' + data_params_dict['save_file_name'] + '_params_dict.pkl', 'wb'), protocol=4)

# Remake the 230125 run to check why the "0" is strangely positioned

In [ ]:
data_params_dict = dill.load(open('../data/params/data_230123_YES_manyObjects_NO_boundaries_NO_tail_ONE_conductivity_params_dict.pkl', 'rb'))

In [ ]:
data_params_dict['save_file_name'] = 'data_230316_YES_manyObjects_NO_boundaries_NO_tail_ONE_conductivity_REMAKE_4'
data_params_dict['worm_resistances']  = data_params_dict['worm_resistances'][9::10]
data_params_dict['worm_capacitances'] = data_params_dict['worm_capacitances'][9::10]
data_params_dict['worm_radii']        = data_params_dict['worm_radii'][[0, 4, 7]]
data_params_dict['worm_position_xs']  = data_params_dict['worm_position_xs'][[4]]  # [[0, 4, 6]]
data_params_dict['worm_position_ys']  = data_params_dict['worm_position_ys'][[0]]  # [[0, 1, 6]]
data_params_dict['worm_position_zs']  = data_params_dict['worm_position_zs'][[2]]  # [[0, 2, 4]]
dill.dump(data_params_dict, open('../data/params/' + data_params_dict['save_file_name'] + '_params_dict.pkl', 'wb'), protocol=4)

### Create dataset for spatial feature extraction

In [ ]:
data_params_dict['save_file_name'] = 'data_230324_YES_manyObjects_NO_boundaries_NO_tail_ONE_conductivity_OnlyFEW_R_C'
data_params_dict['worm_resistances']  = data_params_dict['worm_resistances'][9::10]
data_params_dict['worm_capacitances'] = data_params_dict['worm_capacitances'][9::10]
data_params_dict['worm_radii']        = np.linspace(1e-3, 1.1e-2, 21)[1:]
data_params_dict['worm_position_xs']  = np.linspace(20e-3, -80e-3, 21)[1:]
data_params_dict['worm_position_ys']  = np.linspace(18e-3, 30.5e-3, 26)[:-1]
data_params_dict['worm_position_zs']  = np.linspace(-15e-3, 15e-3, 21)
dill.dump(data_params_dict, open('../data/params/' + data_params_dict['save_file_name'] + '_params_dict.pkl', 'wb'), protocol=4)
data_params_dict